## Autoencoder experiment

1. Build an autoencoder and train it on train data (MIT's)
2. Save the encoded train data (MIT's)
3. Take encoder part from the AE and encode youtube data (the data that will work as "augmenter")
4. Apply some clustering or other algorithm to find similar segments in both encoded datasets (using some MIT videos as seeds)
5. Train NN on the MIT and YT data (those segments that are similar to MIT)
6. Validate the classifier (so you may need to leave some of the MIT and YT data out of the training)
7. Test on our own data

First, we extract features

In [150]:
import os
import pickle
from scipy.io import wavfile
import numpy as np
from librosa import feature
from keras.utils import to_categorical
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import minmax_scale
from sklearn.utils import shuffle


def extract(filename):
    rate, frames = wavfile.read(filename)
    window = 4096
    nfft = 64
    feat = []
    # len(frames)-window
    for i in range(0, len(frames) - window, window):
        feat.append(np.array(frames[i:i + window]))
#         feat.append(np.array(feature.mfcc(frames[i:i + window - 1],
#                                    sr=rate,
#                                    n_fft=nfft,
#                                    hop_length=round(nfft / 2),
#                                    fmax=8000)).flatten())
    feat = np.stack(feat)
    return feat
    
def extract_features(file_wet, file_dry):
    to_replace ="\\/"
    for char in to_replace:
        fw = file_wet.replace(char, "_")
        fd = file_dry.replace(char, "_")
    pickle_file = fw + "-" + fd + ".pkl"
    if os.path.exists(pickle_file):
        print("Using pickle file", pickle_file)
        with open(pickle_file, "rb") as f:
            features, labels = pickle.load(f)
        return features, labels
    features_wet = extract(file_wet)
    features_dry = extract(file_dry)
    labels_wet = np.ones(features_wet.shape[0])
    labels_dry = np.zeros(features_dry.shape[0])
    features = np.concatenate((features_wet, features_dry))
    labels = np.concatenate((labels_wet, labels_dry))
    labels = to_categorical(labels)
    features, labels = shuffle(features, labels, random_state=0)
    # features = minmax_scale(features)
    with open(pickle_file, "wb") as f:
        pickle.dump((features, labels), f, protocol=4)
    return features, labels

In [151]:
from sklearn.preprocessing import StandardScaler, MinMaxScaler


X_1, y_1 = extract_features("dataset/wet1/audio_mono.wav", "dataset/dry1/audio_mono.wav")
X_2, y_2 = extract_features("dataset/wet2/audio_mono.wav", "dataset/dry2/audio_mono.wav")
X_3, y_3 = extract_features("dataset/wet3/audio_mono.wav", "dataset/dry3/audio_mono.wav")
X = np.vstack((X_1, X_2, X_3))
y = np.vstack((y_1, y_2, y_3))

scaler = MinMaxScaler()
scaler.fit(X)
X = scaler.transform(X)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=1)

Using pickle file dataset_wet1_audio_mono.wav-dataset_dry1_audio_mono.wav.pkl
Using pickle file dataset_wet2_audio_mono.wav-dataset_dry2_audio_mono.wav.pkl
Using pickle file dataset_wet3_audio_mono.wav-dataset_dry3_audio_mono.wav.pkl


/media/adilkhan/HDD/Anaconda3/lib/python3.6/site-packages/sklearn/utils/validation.py:475: DataConversionWarning: Data with input dtype int16 was converted to float64 by MinMaxScaler.
  warnings.warn(msg, DataConversionWarning)


Then we train the autoencoder:

In [152]:
from keras.layers import Input, Dense, Dropout
from keras.models import Model


inp = Input(shape=(X_train.shape[1:]))
encoded = Dense(4000, activation='relu')(inp)
encoded = Dropout(0.3)(encoded)
encoded = Dense(2000, activation='relu')(encoded)
encoded = Dropout(0.3)(encoded)
encoded = Dense(1000, activation='relu')(encoded)
encoded = Dropout(0.3)(encoded)
encoded = Dense(200, activation='relu')(encoded)

decoded = Dense(1000, activation='relu')(encoded)
decoded = Dense(2000, activation='relu')(decoded)
decoded = Dense(4000, activation='relu')(decoded)
decoded = Dense(X_train.shape[1], activation='sigmoid')(decoded)

ae = Model(inp, decoded)
ae.compile(loss='mse', optimizer='adam')
ae.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_10 (InputLayer)        (None, 4096)              0         
_________________________________________________________________
dense_69 (Dense)             (None, 4000)              16388000  
_________________________________________________________________
dropout_44 (Dropout)         (None, 4000)              0         
_________________________________________________________________
dense_70 (Dense)             (None, 2000)              8002000   
_________________________________________________________________
dropout_45 (Dropout)         (None, 2000)              0         
_________________________________________________________________
dense_71 (Dense)             (None, 1000)              2001000   
_________________________________________________________________
dropout_46 (Dropout)         (None, 1000)              0         
__________

In [153]:
ae.fit(X_train, X_train, epochs=1000,
       batch_size=128,
       validation_data=(X_test, X_test))

Train on 68187 samples, validate on 17047 samples
Epoch 1/1000
68187/68187 [==============================] - 19s - loss: 0.0011 - val_loss: 1.9291e-04
Epoch 2/1000
68187/68187 [==============================] - 18s - loss: 1.8445e-04 - val_loss: 1.7321e-04
Epoch 3/1000
68187/68187 [==============================] - 18s - loss: 1.8468e-04 - val_loss: 1.7358e-04
Epoch 4/1000
68187/68187 [==============================] - 18s - loss: 1.8497e-04 - val_loss: 1.7381e-04
Epoch 5/1000
68187/68187 [==============================] - 18s - loss: 1.8530e-04 - val_loss: 1.7386e-04
Epoch 6/1000
68187/68187 [==============================] - 18s - loss: 1.8544e-04 - val_loss: 1.7423e-04
Epoch 7/1000
68187/68187 [==============================] - 18s - loss: 1.8577e-04 - val_loss: 1.7361e-04
Epoch 8/1000
68187/68187 [==============================] - 18s - loss: 1.8571e-04 - val_loss: 1.7424e-04
Epoch 9/1000
68187/68187 [==============================] - 18s - loss: 1.8599e-04 - val_loss: 1.7498e-04


68187/68187 [==============================] - 17s - loss: 1.8388e-04 - val_loss: 1.7234e-04
Epoch 78/1000
68187/68187 [==============================] - 17s - loss: 1.8384e-04 - val_loss: 1.7238e-04
Epoch 79/1000
68187/68187 [==============================] - 17s - loss: 1.8382e-04 - val_loss: 1.7247e-04
Epoch 80/1000
68187/68187 [==============================] - 17s - loss: 1.8384e-04 - val_loss: 1.7239e-04
Epoch 81/1000
68187/68187 [==============================] - 17s - loss: 1.8380e-04 - val_loss: 1.7246e-04
Epoch 82/1000
68187/68187 [==============================] - 17s - loss: 1.8383e-04 - val_loss: 1.7236e-04
Epoch 83/1000
68187/68187 [==============================] - 17s - loss: 1.8380e-04 - val_loss: 1.7238e-04
Epoch 84/1000
68187/68187 [==============================] - 17s - loss: 1.8379e-04 - val_loss: 1.7430e-04
Epoch 85/1000
68187/68187 [==============================] - 17s - loss: 1.8384e-04 - val_loss: 1.7226e-04
Epoch 86/1000
68187/68187 [========================

68187/68187 [==============================] - 17s - loss: 1.8370e-04 - val_loss: 1.7223e-04
Epoch 154/1000
68187/68187 [==============================] - 17s - loss: 1.8375e-04 - val_loss: 1.7243e-04
Epoch 155/1000
68187/68187 [==============================] - 17s - loss: 1.8370e-04 - val_loss: 1.7264e-04
Epoch 156/1000
68187/68187 [==============================] - 17s - loss: 1.8373e-04 - val_loss: 1.7226e-04
Epoch 157/1000
68187/68187 [==============================] - 17s - loss: 1.8368e-04 - val_loss: 1.7272e-04
Epoch 158/1000
68187/68187 [==============================] - 17s - loss: 1.8370e-04 - val_loss: 1.7237e-04
Epoch 159/1000
68187/68187 [==============================] - 17s - loss: 1.8371e-04 - val_loss: 1.7227e-04
Epoch 160/1000
68187/68187 [==============================] - 17s - loss: 1.8369e-04 - val_loss: 1.7225e-04
Epoch 161/1000
68187/68187 [==============================] - 17s - loss: 1.8372e-04 - val_loss: 1.7229e-04
Epoch 162/1000
68187/68187 [===============

68187/68187 [==============================] - 17s - loss: 1.8365e-04 - val_loss: 1.7223e-04
Epoch 230/1000
68187/68187 [==============================] - 17s - loss: 1.8366e-04 - val_loss: 1.7222e-04
Epoch 231/1000
68187/68187 [==============================] - 17s - loss: 1.8368e-04 - val_loss: 1.7243e-04
Epoch 232/1000
68187/68187 [==============================] - 17s - loss: 1.8365e-04 - val_loss: 1.7218e-04
Epoch 233/1000
68187/68187 [==============================] - 17s - loss: 1.8368e-04 - val_loss: 1.7313e-04
Epoch 234/1000
68187/68187 [==============================] - 17s - loss: 1.8366e-04 - val_loss: 1.7219e-04
Epoch 235/1000
68187/68187 [==============================] - 17s - loss: 1.8365e-04 - val_loss: 1.7319e-04
Epoch 236/1000
68187/68187 [==============================] - 17s - loss: 1.8369e-04 - val_loss: 1.7222e-04
Epoch 237/1000
68187/68187 [==============================] - 17s - loss: 1.8366e-04 - val_loss: 1.7225e-04
Epoch 238/1000
68187/68187 [===============

68187/68187 [==============================] - 17s - loss: 1.8363e-04 - val_loss: 1.7218e-04
Epoch 306/1000
68187/68187 [==============================] - 17s - loss: 1.8363e-04 - val_loss: 1.7243e-04
Epoch 307/1000
68187/68187 [==============================] - 17s - loss: 1.8361e-04 - val_loss: 1.7222e-04
Epoch 308/1000
68187/68187 [==============================] - 17s - loss: 1.8367e-04 - val_loss: 1.7217e-04
Epoch 309/1000
68187/68187 [==============================] - 17s - loss: 1.8363e-04 - val_loss: 1.7237e-04
Epoch 310/1000
68187/68187 [==============================] - 17s - loss: 1.8361e-04 - val_loss: 1.7224e-04
Epoch 311/1000
68187/68187 [==============================] - 17s - loss: 1.8366e-04 - val_loss: 1.7213e-04
Epoch 312/1000
68187/68187 [==============================] - 17s - loss: 1.8363e-04 - val_loss: 1.7219e-04
Epoch 313/1000
68187/68187 [==============================] - 17s - loss: 1.8362e-04 - val_loss: 1.7234e-04
Epoch 314/1000
68187/68187 [===============

...

68187/68187 [==============================] - 17s - loss: 1.8355e-04 - val_loss: 1.7216e-04
Epoch 686/1000
68187/68187 [==============================] - 17s - loss: 1.8354e-04 - val_loss: 1.7207e-04
Epoch 687/1000
68187/68187 [==============================] - 17s - loss: 1.8351e-04 - val_loss: 1.7206e-04
Epoch 688/1000
68187/68187 [==============================] - 17s - loss: 1.8355e-04 - val_loss: 1.7217e-04
Epoch 689/1000
68187/68187 [==============================] - 17s - loss: 1.8353e-04 - val_loss: 1.7214e-04
Epoch 690/1000
68187/68187 [==============================] - 17s - loss: 1.8354e-04 - val_loss: 1.7209e-04
Epoch 691/1000
68187/68187 [==============================] - 17s - loss: 1.8357e-04 - val_loss: 1.7209e-04
Epoch 692/1000
68187/68187 [==============================] - 17s - loss: 1.8351e-04 - val_loss: 1.7211e-04
Epoch 693/1000
68187/68187 [==============================] - 17s - loss: 1.8353e-04 - val_loss: 1.7207e-04
Epoch 694/1000
68187/68187 [===============

68187/68187 [==============================] - 17s - loss: 1.8352e-04 - val_loss: 1.7262e-04
Epoch 762/1000
68187/68187 [==============================] - 17s - loss: 1.8354e-04 - val_loss: 1.7207e-04
Epoch 763/1000
68187/68187 [==============================] - 17s - loss: 1.8350e-04 - val_loss: 1.7208e-04
Epoch 764/1000
68187/68187 [==============================] - 17s - loss: 1.8353e-04 - val_loss: 1.7223e-04
Epoch 765/1000
68187/68187 [==============================] - 17s - loss: 1.8354e-04 - val_loss: 1.7211e-04
Epoch 766/1000
68187/68187 [==============================] - 17s - loss: 1.8351e-04 - val_loss: 1.7208e-04
Epoch 767/1000
68187/68187 [==============================] - 17s - loss: 1.8353e-04 - val_loss: 1.7208e-04
Epoch 768/1000
68187/68187 [==============================] - 17s - loss: 1.8352e-04 - val_loss: 1.7233e-04
Epoch 769/1000
68187/68187 [==============================] - 17s - loss: 1.8352e-04 - val_loss: 1.7215e-04
Epoch 770/1000
68187/68187 [===============

68187/68187 [==============================] - 17s - loss: 1.8349e-04 - val_loss: 1.7208e-04
Epoch 838/1000
68187/68187 [==============================] - 17s - loss: 1.8351e-04 - val_loss: 1.7209e-04
Epoch 839/1000
68187/68187 [==============================] - 17s - loss: 1.8352e-04 - val_loss: 1.7206e-04
Epoch 840/1000
68187/68187 [==============================] - 17s - loss: 1.8353e-04 - val_loss: 1.7243e-04
Epoch 841/1000
68187/68187 [==============================] - 17s - loss: 1.8352e-04 - val_loss: 1.7207e-04
Epoch 842/1000
68187/68187 [==============================] - 17s - loss: 1.8351e-04 - val_loss: 1.7207e-04
Epoch 843/1000
68187/68187 [==============================] - 17s - loss: 1.8353e-04 - val_loss: 1.7220e-04
Epoch 844/1000
68187/68187 [==============================] - 17s - loss: 1.8349e-04 - val_loss: 1.7225e-04
Epoch 845/1000
68187/68187 [==============================] - 17s - loss: 1.8353e-04 - val_loss: 1.7211e-04
Epoch 846/1000
68187/68187 [===============

68187/68187 [==============================] - 17s - loss: 1.8351e-04 - val_loss: 1.7212e-04
Epoch 914/1000
68187/68187 [==============================] - 17s - loss: 1.8348e-04 - val_loss: 1.7208e-04
Epoch 915/1000
68187/68187 [==============================] - 17s - loss: 1.8351e-04 - val_loss: 1.7216e-04
Epoch 916/1000
68187/68187 [==============================] - 17s - loss: 1.8348e-04 - val_loss: 1.7366e-04
Epoch 917/1000
68187/68187 [==============================] - 17s - loss: 1.8350e-04 - val_loss: 1.7209e-04
Epoch 918/1000
68187/68187 [==============================] - 17s - loss: 1.8348e-04 - val_loss: 1.7258e-04
Epoch 919/1000
68187/68187 [==============================] - 17s - loss: 1.8348e-04 - val_loss: 1.7217e-04
Epoch 920/1000
68187/68187 [==============================] - 17s - loss: 1.8350e-04 - val_loss: 1.7214e-04
Epoch 921/1000
68187/68187 [==============================] - 17s - loss: 1.8351e-04 - val_loss: 1.7210e-04
Epoch 922/1000
68187/68187 [===============

68187/68187 [==============================] - 17s - loss: 1.8348e-04 - val_loss: 1.7209e-04
Epoch 990/1000
68187/68187 [==============================] - 17s - loss: 1.8351e-04 - val_loss: 1.7214e-04
Epoch 991/1000
68187/68187 [==============================] - 17s - loss: 1.8349e-04 - val_loss: 1.7207e-04
Epoch 992/1000
68187/68187 [==============================] - 17s - loss: 1.8348e-04 - val_loss: 1.7207e-04
Epoch 993/1000
68187/68187 [==============================] - 17s - loss: 1.8348e-04 - val_loss: 1.7622e-04
Epoch 994/1000
68187/68187 [==============================] - 18s - loss: 1.8351e-04 - val_loss: 1.7207e-04
Epoch 995/1000
68187/68187 [==============================] - 18s - loss: 1.8349e-04 - val_loss: 1.7215e-04
Epoch 996/1000
68187/68187 [==============================] - 18s - loss: 1.8348e-04 - val_loss: 1.7207e-04
Epoch 997/1000
68187/68187 [==============================] - 18s - loss: 1.8348e-04 - val_loss: 1.7215e-04.839
Epoch 998/1000
68187/68187 [===========

In [154]:
encoder = Model(inp, encoded)
encoder.compile(loss='mse', optimizer='adam')
encoder.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_10 (InputLayer)        (None, 4096)              0         
_________________________________________________________________
dense_69 (Dense)             (None, 4000)              16388000  
_________________________________________________________________
dropout_44 (Dropout)         (None, 4000)              0         
_________________________________________________________________
dense_70 (Dense)             (None, 2000)              8002000   
_________________________________________________________________
dropout_45 (Dropout)         (None, 2000)              0         
_________________________________________________________________
dense_71 (Dense)             (None, 1000)              2001000   
_________________________________________________________________
dropout_46 (Dropout)         (None, 1000)              0         
__________

In [155]:
# serialize model to JSON
encoder_json = encoder.to_json()
with open("model.json", "w") as json_file:
    json_file.write(encoder_json)
# serialize weights to HDF5
encoder.save_weights("model.h5")
print("Saved model to disk")

Saved model to disk


In [14]:
from keras.models import model_from_json

json_file = open('model.json', 'r')
loaded_model_json = json_file.read()
json_file.close()
loaded_model = model_from_json(loaded_model_json)
# load weights into new model
loaded_model.load_weights("model.h5")
print("Loaded model from disk")

Loaded model from disk


In [156]:
X_yt, y_yt = extract_features("yt_data/wet_mono.wav", "yt_data/dry_mono.wav")
X_yt_train, X_yt_test, y_yt_train, y_yt_test = train_test_split(X_yt, y_yt, test_size=0.2, random_state=1)
X_yt_train = scaler.transform(X_yt_train)
X_yt_test = scaler.transform(X_yt_test)

Using pickle file yt_data_wet_mono.wav-yt_data_dry_mono.wav.pkl


In [161]:
enc_train = encoder.predict(X_yt_train)
enc_test = encoder.predict(X_yt_test)
enc_train_mit = encoder.predict(X_train)
enc_test_mit = encoder.predict(X_test)
X_new = np.vstack((enc_train, enc_train_mit))
X_test_new = np.vstack((enc_test, enc_test_mit))
y_new = np.vstack((y_train, y_yt_train))
y_test_new = np.vstack((y_test, y_yt_test))

In [162]:
with open("encoded.pkl", "wb") as f:
    pickle.dump((X_new, X_test_new, y_new, y_test_new), f, protocol=4)

In [182]:
with open("encoded.pkl", "rb") as f:
    X_new, X_test_new, y_new, y_test_new = pickle.load(f)

In [175]:
X_new.shape

(626795, 200)

In [183]:
X_new = np.expand_dims(X_new, axis=1)
X_new = X_new.reshape((X_new.shape[0],X_new.shape[2], 1))

X_test_new = np.expand_dims(X_test_new, axis=1)
X_test_new = X_test_new.reshape((X_test_new.shape[0], X_test_new.shape[2], 1))

In [184]:
X_test_new.shape

(156700, 200, 1)

In [185]:
from time import time
from keras.models import Sequential
from keras.layers import Conv1D, MaxPooling1D, GlobalAveragePooling1D, Dropout, Dense, Flatten, LSTM
from keras.layers.wrappers import Bidirectional, TimeDistributed
from keras.callbacks import Callback
from datetime import datetime
import os

dt = datetime.now().strftime("%d-%m-%Y.%H-%M")

class TestCallback(Callback):
    def __init__(self, test_data, number):
        self.test_data = test_data
        self.number = number

    def on_epoch_end(self, epoch, logs={}):
        x, y = self.test_data
        loss, acc = self.model.evaluate(x, y, verbose=0)
        log_filename = "log." + dt + ".csv"
        with open(log_filename, "a") as log:
            log.write("{},{},{},{},{}\n".format(self.number, epoch, loss, acc, logs["acc"]))

def def_model_cnn_blstm(input_shape):
    model = Sequential()
    model.add(TimeDistributed(Conv1D(filters=64, kernel_size=64, strides=1, padding="same", activation='tanh'), input_shape=input_shape))
    model.add(TimeDistributed(MaxPooling1D(2)))
    model.add(TimeDistributed(Dropout(0.4)))
    # model.add(TimeDistributed(Conv1D(32, 32, activation='relu')))
    # model.add(TimeDistributed(MaxPooling1D(4)))
    # model.add(TimeDistributed(Dropout(0.3)))
    model.add(TimeDistributed(Conv1D(64, 64, padding="same", activation='tanh')))
    model.add(TimeDistributed(MaxPooling1D(2)))
    model.add(TimeDistributed(Dropout(0.2)))
    model.add(TimeDistributed(Conv1D(64, 64, padding="same", activation='tanh')))
    model.add(TimeDistributed(MaxPooling1D(2)))
    model.add(TimeDistributed(Dropout(0.2)))
    # model.add(TimeDistributed(Conv1D(64, 16, activation='relu')))
    # model.add(TimeDistributed(MaxPooling1D(4)))
    # model.add(TimeDistributed(Dropout(0.4)))
    model.add(TimeDistributed(Conv1D(128, 64, padding="same", activation='tanh')))
    model.add(TimeDistributed(MaxPooling1D(2)))
    model.add(TimeDistributed(Dropout(0.2)))
    model.add(TimeDistributed(Conv1D(128, 64, padding="same", activation='tanh')))
    model.add(TimeDistributed(MaxPooling1D(2)))
    # model.add(TimeDistributed(Dropout(0.4)))
    model.add(TimeDistributed(Conv1D(128, 64, padding="same", activation='tanh')))
    model.add(TimeDistributed(MaxPooling1D(2)))
    # model.add(TimeDistributed(Dropout(0.4)))
    model.add(TimeDistributed(Conv1D(256, 64, padding="same", activation='tanh')))
    model.add(TimeDistributed(MaxPooling1D(2)))
    # model.add(TimeDistributed(Dropout(0.4)))
    model.add(TimeDistributed(Conv1D(256, 64, padding="same", activation='tanh')))
    model.add(TimeDistributed(GlobalAveragePooling1D()))
    model.add(Bidirectional(LSTM(256)))
    model.add(Dense(2, activation='softmax'))
    model.compile(loss='categorical_crossentropy',
                  optimizer='adam',
                  metrics=['accuracy'])
    model.summary()
    return model


def def_model(input_shape):
    model = Sequential()
    model.add(Conv1D(filters=8, kernel_size=8, strides=2, padding="same",
                     input_shape=input_shape, kernel_initializer='uniform',
                     activation='relu'))
    #model.add(Dropout(0.5))
    model.add(Conv1D(32, 8, padding="same", activation='tanh'))
    model.add(MaxPooling1D(4))
    model.add(Dropout(0.5))
    model.add(Conv1D(32, 8, padding="same", activation='tanh'))
    # model.add(MaxPooling1D(4))
    # model.add(Flatten())
    model.add(GlobalAveragePooling1D())
    # model.add(Dropout(0.5))
    model.add(Dense(64, activation='relu'))
    model.add(Dropout(0.5))
    model.add(Dense(2, activation='softmax'))
    model.compile(loss='categorical_crossentropy',
                  optimizer='adam',
                  metrics=['accuracy'])
    model.summary()
    return model

def model2(input_shape):
    model = Sequential()
    model.add(Dense(1000, activation='tanh', input_shape=input_shape))
    model.add(Dense(1000, activation='tanh'))
    model.add(Dense(1000, activation='tanh'))
    model.add(Dense(2, activation='softmax'))
    model.compile(loss='categorical_crossentropy',
                  optimizer='adam',
                  metrics=['accuracy'])
    model.summary()
    return model



model = def_model(X_new.shape[1:])
# model = def_model_cnn_blstm(enc_train.shape[1:])
print("Dataset shape:", X_new.shape)

testCallback0 = TestCallback((X_test_new, y_test_new), 1)

model.fit(X_new, y_new, validation_data=(X_test_new, y_test_new),
          batch_size=128, epochs=75, verbose=1)
          #callbacks=[testCallback0])



_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv1d_70 (Conv1D)           (None, 100, 8)            72        
_________________________________________________________________
conv1d_71 (Conv1D)           (None, 100, 32)           2080      
_________________________________________________________________
max_pooling1d_46 (MaxPooling (None, 25, 32)            0         
_________________________________________________________________
dropout_47 (Dropout)         (None, 25, 32)            0         
_________________________________________________________________
conv1d_72 (Conv1D)           (None, 25, 32)            8224      
_________________________________________________________________
global_average_pooling1d_9 ( (None, 32)                0         
_________________________________________________________________
dense_97 (Dense)             (None, 64)                2112      
__________

626795/626795 [==============================] - 18s - loss: 0.6883 - acc: 0.5492 - val_loss: 0.6890 - val_acc: 0.5466
Epoch 52/75
626795/626795 [==============================] - 18s - loss: 0.6883 - acc: 0.5492 - val_loss: 0.6888 - val_acc: 0.5466
Epoch 53/75
626795/626795 [==============================] - 18s - loss: 0.6883 - acc: 0.5492 - val_loss: 0.6888 - val_acc: 0.5466
Epoch 54/75
626795/626795 [==============================] - 18s - loss: 0.6883 - acc: 0.5492 - val_loss: 0.6888 - val_acc: 0.5466
Epoch 55/75
626795/626795 [==============================] - 18s - loss: 0.6883 - acc: 0.5492 - val_loss: 0.6888 - val_acc: 0.5466
Epoch 56/75
626795/626795 [==============================] - 18s - loss: 0.6883 - acc: 0.5492 - val_loss: 0.6888 - val_acc: 0.5466
Epoch 57/75
626795/626795 [==============================] - 18s - loss: 0.6883 - acc: 0.5492 - val_loss: 0.6888 - val_acc: 0.5466
Epoch 58/75
626795/626795 [==============================] - 18s - loss: 0.6883 - acc: 0.5492 -